In [1]:
# !pip install llama-index
# !pip install llama-index-llms-ollama
# !pip install llama-index-llms-replicate

# !pip install llama-index-embeddings-huggingface
# %pip install llama-index-embeddings-ollama

# !pip install llama-parse
# !pip install replicate

# %pip install tf-keras # for huggingfaceembedding - which needs older keras

# %pip install llama-index-vector-stores-faiss

In [7]:
# https://medium.com/@sakthiveltvt.thangaraj/introduction-to-nest-asyncio-for-python-developers-afd7bed44768

import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3")#, #request_timeout=120.0)

In [9]:
# needs older keras
'''
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. 
Please install the backwards-compatible tf-keras package with `pip install tf-keras`
'''
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

In [19]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# The default chunk size is 1024, while the default chunk overlap is 20 causes timeout on large documents
Settings.chunk_size = 256 
Settings.chunk_overlap = 50

In [11]:
import faiss
#https://huggingface.co/BAAI/bge-small-en-v1.5
# dimensions of BAAI/bge-large-en-v1.5
d = 1024
faiss_index = faiss.IndexFlatL2(d)

In [12]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [8]:
# # load documents
# documents = SimpleDirectoryReader("../GenAI/data/Twelfthphysics1/").load_data()
# #documents

# # https://docs.llamaindex.ai/en/stable/examples/vector_stores/FaissIndexDemo/
# vector_store = FaissVectorStore(faiss_index=faiss_index)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context
# )

# save index to disk
# index.storage_context.persist("../GenAI/data/llamaindex/vectorstores/faiss/twelfthphysics1/")

In [13]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("../GenAI/data/llamaindex/vectorstores/faiss/twelfthphysics1/")

storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="../GenAI/data/llamaindex/vectorstores/faiss/twelfthphysics1/"
)

index = load_index_from_storage(storage_context=storage_context)

In [20]:
engine = index.as_query_engine(similarity_top_k=3)

In [11]:
response = engine.query("What is DISPLACEMENT CURRENT? answer from available context, dont generate from your own")

In [12]:
print(response)

The displacement current has the same physical effects as the conduction current. In some cases, for example, steady electric fields in a conducting wire, the displacement current may be zero since the electric field E does not change with time. In other cases, for example, the charging capacitor above, both conduction and displacement currents may be present in different regions of space.


In [13]:
print(response)

The displacement current has the same physical effects as the conduction current. In some cases, for example, steady electric fields in a conducting wire, the displacement current may be zero since the electric field E does not change with time. In other cases, for example, the charging capacitor above, both conduction and displacement currents may be present in different regions of space.


In [21]:
def predict(prompt):
    completion = engine.query(prompt)
    return completion

In [1]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
async def chatbot_response(user_input):
    if 'hello' in user_input.lower():
        return "Hello, How can I assist you today ?"
    elif 'how' in user_input.lower():
        return 'Sure, Let me fetch the results and present it to you.'
    else:
        return "Sorry, I didn't understand your request."
    
async def chat():
    print("Hi, I'm your assistant, type 'bye' to exit")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "bye":
            print("Chatbot: Bye, Have a great day.")
            break
        response = await chatbot_response(user_input)
        print(f'chatbot response:{response}')

#start the chat
asyncio.run(chat())

Hi, I'm your assistant, type 'bye' to exit
chatbot response:Sure, Let me fetch the results and present it to you.
chatbot response:Hello, How can I assist you today ?
chatbot response:Sorry, I didn't understand your request.
chatbot response:Sorry, I didn't understand your request.
Chatbot: Bye, Have a great day.


# Gradio

In [3]:
%pip install gradio

   ---------------------------------------- 0.0/57.1 MB ? eta -:--:--
   - -------------------------------------- 1.6/57.1 MB 7.6 MB/s eta 0:00:08
   --- ------------------------------------ 5.2/57.1 MB 12.3 MB/s eta 0:00:05
   ------ --------------------------------- 8.7/57.1 MB 14.1 MB/s eta 0:00:04
   -------- ------------------------------- 12.3/57.1 MB 15.1 MB/s eta 0:00:03
   ----------- ---------------------------- 16.3/57.1 MB 15.8 MB/s eta 0:00:03
   ------------- -------------------------- 19.7/57.1 MB 15.7 MB/s eta 0:00:03
   ---------------- ----------------------- 23.3/57.1 MB 16.1 MB/s eta 0:00:03
   ------------------- -------------------- 27.3/57.1 MB 16.3 MB/s eta 0:00:02
   ---------------------- ----------------- 31.5/57.1 MB 16.6 MB/s eta 0:00:02
   ---------------------- ----------------- 32.5/57.1 MB 15.4 MB/s eta 0:00:02
   ------------------------ --------------- 34.3/57.1 MB 15.1 MB/s eta 0:00:02
   ------------------------- -------------- 37.0/57.1 MB 14.9 MB/


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import gradio as gr


def greet(game):
    return "Hello " + game

p="What is DISPLACEMENT CURRENT? answer from available context, dont generate from your own"
demo = gr.Interface(fn=predict, inputs="text", outputs="text")

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\rachaudhari\OneDrive - Deloitte (O365D)\General\pythonenvs\.venv\Lib\site-packages\httpx\_transports\default.py", line 72, in map_httpcore_exceptions
    yield
  File "c:\Users\rachaudhari\OneDrive - Deloitte (O365D)\General\pythonenvs\.venv\Lib\site-packages\httpx\_transports\default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rachaudhari\OneDrive - Deloitte (O365D)\General\pythonenvs\.venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "c:\Users\rachaudhari\OneDrive - Deloitte (O365D)\General\pythonenvs\.venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rachaudhari\OneDrive - Deloitte (O365D)\General\pythonenvs\.venv\Lib\site-packages\httpcore\